In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import win32com.client
import os

params

In [188]:
VISUM_PATH = os.path.join(os.getcwd(),"data//visum.ver")
POI_CAT = 1
MTX_JRT = 20
MTX_PRT = 11

CZAS_PARKOWANIA = {"SPPN":6*60,
                   "POZA_SPPN":3*60}

In [189]:
Visum = win32com.client.Dispatch("Visum.Visum")  # uruchom Visum
Visum.LoadVersion(VISUM_PATH)  # zaladuj plik

In [202]:
POIs = pd.DataFrame(list(Visum.Net.POICategories.ItemByKey(POI_CAT).POIs.GetMultipleAttributes(["No", "ZoneID","Czas_Zone"])),
                    columns = ["POI", "ZoneID","Czas_Zone"])
POIs.ZoneID = POIs.ZoneID.astype(int)
POIs.POI = POIs.POI.astype(int)

In [203]:
cols = Visum.Net.Zones.GetMultiAttValues('No', False)
cols = [int(_[1]) for _ in cols]
mtx = Visum.Net.Matrices.ItemByKey(MTX_JRT)
vals = list(mtx.GetValuesFloat())
JRT = pd.DataFrame(vals, cols, cols)
mtx = Visum.Net.Matrices.ItemByKey(MTX_PRT)
vals = list(mtx.GetValuesFloat())
PRT = pd.DataFrame(vals, cols, cols)

In [204]:
JRT = JRT.stack().reset_index()
JRT.columns = ['Z',"Do","Czas_PuT"]
PRT = PRT.stack().reset_index()
PRT.columns = ['Z',"Do","Czas_PrT"]

In [205]:
jk = pd.merge(JRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()
ki = pd.merge(POIs,JRT,how='outer',left_on="ZoneID",right_on="Z").dropna()
jki_JRT = pd.merge(jk,ki,on='POI')

In [206]:
jki_JRT['Czas_PuT']=jki_JRT.Czas_PuT_x + jki_JRT.Czas_PuT_y + 2*jki_JRT.Czas_Zone_x  
jki_JRT = jki_JRT[['Z_x','POI','Do_y','Czas_PuT']]
jki_JRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PuT']

In [207]:
jk = pd.merge(PRT,POIs,how='outer',left_on="Do",right_on="ZoneID").dropna()
ki = pd.merge(POIs,PRT,how='outer',left_on="ZoneID",right_on="Z").dropna()
jki_PRT = pd.merge(jk,ki,on='POI')

In [208]:
jki_PRT['Czas_PrT']=jki_PRT.Czas_PrT_x + jki_PRT.Czas_PrT_y + 2*jki_PRT.Czas_Zone_x + 2*CZAS_PARKOWANIA['SPPN'] 
jki_PRT = jki_PRT[['Z_x','POI','Do_y','Czas_PrT']]
jki_PRT.columns = ['Z_Rejon','POI','Do_Rejon','Czas_PrT']

In [211]:
jki = pd.merge(jki_PRT,jki_JRT,on=['Z_Rejon','POI','Do_Rejon'])
jki.to_csv("data//From_Via_To.csv")

In [210]:
jki.head(1)

,Z_Rejon,POI,Do_Rejon,Czas_PrT,Czas_PuT
0,1,15.00,1,876.23,156.23
